In [5]:
import pandas as pd
import numpy as np
import sqlite3

In [264]:
df = pd.read_csv("scraped_data.csv")

In [6]:
def get_data():
    cnx = sqlite3.connect('it-job-market.db')
    df = pd.read_sql_query("SELECT * FROM offers", cnx)
    cnx.commit()
    cnx.close()
    return df

In [7]:
df = get_data()
df

,job_title,employer,price_from,price_to,price_unit,url,contract_type,address,city,expiration_date
0,Junior / Mid Service Desk Specialist,ASTEK Polska,5000,7000,gross / mth.,https://www.pracuj.pl/praca/junior-mid-service...,B2B,"Mokotowska 1, Śródmieście, Warszawa","Warszawa, Masovian",10-09-2023
1,Programista / Programistka Prestashop,InSystem Sp. z o.o.,7000,10000,gross / mth.,https://www.pracuj.pl/praca/programista-progra...,employment,"Kijowska 1, Warszawa","Warszawa, mazowieckie",24-09-2023
2,Mid DevOps Engineer (Linux),Centuria S.A.,15000,18000,gross / mth.,https://www.pracuj.pl/praca/mid-devops-enginee...,B2B,"Sarmacka 11, Stare Miasto, Poznań","Poznań, wielkopolskie",28-08-2023
3,Oracle Apex Developer,JellyTech,15000,19300,gross / mth.,https://www.pracuj.pl/praca/oracle-apex-develo...,B2B,"Encyklopedyczna 2A, Bielany, Warszawa","Warszawa, mazowieckie",01-09-2023
4,Python Developer,JellyTech,15000,18400,gross / mth.,https://www.pracuj.pl/praca/python-developer-k...,B2B,None,"Kraków, małopolskie",01-09-2023
5,Backend .NET Engineer,DIGITEUM sp. z o.o.,15000,16700,gross / mth.,https://www.pracuj.pl/praca/backend-net-engine...,"employment, B2B","Szafarnia 11, Gdańsk","Gdańsk, Pomeranian",05-09-2023
6,BI Developer (NiFi/Airflow),JellyTech,14000,17600,gross / mth.,https://www.pracuj.pl/praca/bi-developer-nifi-...,B2B,"Encyklopedyczna 2A, Bielany, Warszawa","Warszawa, mazowieckie",28-08-2023
7,Software Engineer (SaaS Front-end AWS),Adtran,12000,17000,gross / mth.,https://www.pracuj.pl/praca/software-engineer-...,employment,"Łużycka 8c, Mały Kack, Gdynia","Gdynia, Pomeranian",28-08-2023
8,Java Developer,JellyTech,16000,25000,gross / mth.,https://www.pracuj.pl/praca/java-developer-war...,B2B,"Encyklopedyczna 2A, Bielany, Warszawa","Warszawa, mazowieckie",28-08-2023
9,Fullstack Developer,JellyTech,12000,20000,gross / mth.,https://www.pracuj.pl/praca/fullstack-develope...,B2B,"Encyklopedyczna 2A, Bielany, Warszawa","Warszawa, mazowieckie",28-08-2023


In [12]:
from geopy.extra.rate_limiter import RateLimiter
from geopy import Nominatim

locator = Nominatim(user_agent='myGeocoder')

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['address'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [13]:
df.head()

,job_title,employer,price_from,price_to,price_unit,url,contract_type,address,city,expiration_date,location,point,latitude,longitude,altitude
0,Junior / Mid Service Desk Specialist,ASTEK Polska,5000,7000,gross / mth.,https://www.pracuj.pl/praca/junior-mid-service...,B2B,"Mokotowska 1, Śródmieście, Warszawa","Warszawa, Masovian",10-09-2023,"(Boston Consulting Group, 1, Mokotowska, Marsz...","(52.2175887, 21.016048, 0.0)",52.217589,21.016048,0.0
1,Programista / Programistka Prestashop,InSystem Sp. z o.o.,7000,10000,gross / mth.,https://www.pracuj.pl/praca/programista-progra...,employment,"Kijowska 1, Warszawa","Warszawa, mazowieckie",24-09-2023,"(Millennium Bank, 1, Kijowska, Port Praski, St...","(52.2504598, 21.0438381, 0.0)",52.250460,21.043838,0.0
2,Mid DevOps Engineer (Linux),Centuria S.A.,15000,18000,gross / mth.,https://www.pracuj.pl/praca/mid-devops-enginee...,B2B,"Sarmacka 11, Stare Miasto, Poznań","Poznań, wielkopolskie",28-08-2023,None,None,NaN,NaN,NaN
3,Oracle Apex Developer,JellyTech,15000,19300,gross / mth.,https://www.pracuj.pl/praca/oracle-apex-develo...,B2B,"Encyklopedyczna 2A, Bielany, Warszawa","Warszawa, mazowieckie",01-09-2023,"(2A, Encyklopedyczna, Młociny, Bielany, Warsza...","(52.303703549999994, 20.929381906502584, 0.0)",52.303704,20.929382,0.0
4,Python Developer,JellyTech,15000,18400,gross / mth.,https://www.pracuj.pl/praca/python-developer-k...,B2B,None,"Kraków, małopolskie",01-09-2023,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)",44.933143,7.540121,0.0


In [266]:
df.shape

(4132, 5)

In [267]:
df.describe()

,employer,job_title,price_from,price_to,price_unit
count,4132,4132,1028,1071,1071
unique,1519,3279,154,195,8
top,Capgemini Polska,Java Developer,10 000,20 000,brutto / mies.
freq,81,33,72,61,283


In [307]:
df.dtypes

employer      string[python]
job_title     string[python]
price_from           Float64
price_to               Int64
price_unit    string[python]
dtype: object

In [269]:
def repl(x):
    if isinstance(x,str):
        output = ""
        for char in x:
            if char == ',':
                output += "."
                continue
            try:
                int(char)
                output += char
            except:
                pass
        return float(output)
#         return x.replace("&nbsp", "x")
    else:
        return x

df['price_from'] = df['price_from'].apply(repl)
df['price_to'] = df['price_to'].apply(repl)

In [82]:
df = df.infer_objects()
df.dtypes

employer      string[python]
job_title     string[python]
price_from    string[python]
price_to      string[python]
price_unit    string[python]
dtype: object

In [306]:
df = df.convert_dtypes()

In [28]:
# using dictionary to convert specific columns
convert_dict = {'employer': str,
                'job_title': str,
                'price_from': str,
                'price_to': str,
                'price_unit': str,
                }
 
df = df.astype(convert_dict)
print(df.dtypes)

employer      object
job_title     object
price_from    object
price_to      object
price_unit    object
dtype: object


In [162]:
df.sort_values(by=['price_to'], ascending=False)

,employer,job_title,price_from,price_to,price_unit
274,ITDS Polska Sp. z o.o.,Database Developer (Oracle),55000.0,70000.0,net (+ VAT) / mth.
267,ITDS Polska Sp. z o.o.,Java Developer (Micro-Services),55000.0,70000.0,net (+ VAT) / mth.
287,ITDS Polska Sp. z o.o.,Senior Java Developer,55000.0,70000.0,net (+ VAT) / mth.
301,ITDS Polska Sp. z o.o.,QA,55000.0,70000.0,net (+ VAT) / mth.
261,ITDS Polska Sp. z o.o.,Senior Backup Engineer,46200.0,54600.0,net (+ VAT) / mth.
...,...,...,...,...,...
4125,GynCentrum Sp. z o.o.,Specjalista ds. systemów zarządzania,NaN,NaN,<NA>
4127,Ingersoll Rand,Area Sales Manager,NaN,NaN,<NA>
4129,Arche Consulting Sp z o.o.,Specjalista ds. wsparcia IT z j. niemieckim,NaN,NaN,<NA>
4130,Sescom S.A.,Specjalista helpdesk z j. słowackim i czeskim,NaN,NaN,<NA>


In [195]:
df.nlargest(20, 'price_from')

,employer,job_title,price_from,price_to,price_unit
267,ITDS Polska Sp. z o.o.,Java Developer (Micro-Services),55000.0,70000.0,net (+ VAT) / mth.
274,ITDS Polska Sp. z o.o.,Database Developer (Oracle),55000.0,70000.0,net (+ VAT) / mth.
287,ITDS Polska Sp. z o.o.,Senior Java Developer,55000.0,70000.0,net (+ VAT) / mth.
301,ITDS Polska Sp. z o.o.,QA,55000.0,70000.0,net (+ VAT) / mth.
261,ITDS Polska Sp. z o.o.,Senior Backup Engineer,46200.0,54600.0,net (+ VAT) / mth.
1068,ITDS Polska Sp. z o.o.,CMDB Librarian (Splunk),46200.0,54600.0,net (+ VAT) / mth.
1070,ITDS Polska Sp. z o.o.,Splunk Engineer,46200.0,54600.0,net (+ VAT) / mth.
1603,NTIATIVE sp. z o.o.,Lead DevOps,35000.0,40000.0,net (+ VAT) / mth.
3929,TASKRABBIT POLAND sp. z o.o.,Full Stack Software Engineer,33711.0,45584.0,brutto / mies.
203,7N Sp. z o.o.,Senior Cloud Engineer,33600.0,36960.0,netto (+ VAT) / mies.


In [302]:
df[['employer','price_unit']].groupby(['price_unit']).count()

,employer
price_unit,
gross / hr.,192
gross / mth.,836


In [301]:
# df.loc[df['price_unit'] == 'brutto / mies.', 'price_unit'] = 'gross / mth.'
df['price_unit'] = np.where(df['price_unit'] == ' brutto / mies.', 'gross / mth.', df['price_unit'])
df['price_unit'] = np.where(df['price_unit'] == ' net (+ VAT) / mth.', 'gross / mth.', df['price_unit'])
df['price_unit'] = np.where(df['price_unit'] == ' netto (+ VAT) / mies.', 'gross / mth.', df['price_unit'])
df['price_unit'] = np.where(df['price_unit'] == ' netto (+ VAT) / mies.', 'gross / mth.', df['price_unit'])
df['price_unit'] = np.where(df['price_unit'] == ' gross / mth.', 'gross / mth.', df['price_unit'])

df['price_unit'] = np.where(df['price_unit'] == ' brutto / godz.', 'gross / hr.', df['price_unit'])
df['price_unit'] = np.where(df['price_unit'] == ' net (+ VAT) / hr.', 'gross / hr.', df['price_unit'])
df['price_unit'] = np.where(df['price_unit'] == ' netto (+ VAT) / godz.', 'gross / hr.', df['price_unit'])
df['price_unit'] = np.where(df['price_unit'] == ' gross / hr.', 'gross / hr.', df['price_unit'])

In [304]:
df.head()

,employer,job_title,price_from,price_to,price_unit
0,Prime Engineering Poland,Senior Test Automation Engineer,17000.0,21000,gross / mth.
1,Velis RET spółka z ograniczoną odpowiedzialnością,Project Management Specialist,5000.0,7000,gross / mth.
2,Velis RET spółka z ograniczoną odpowiedzialnością,Project Manager,7000.0,10000,gross / mth.
3,4IT SOLUTIONS sp. z o.o.,Java Fullstack Developer,90.0,130,gross / hr.
4,ITFS sp. z o.o.,Tester Automatyzujący,80.0,120,gross / hr.


In [241]:
df = df.fillna(np.nan)

In [233]:
pd.NA

<NA>

In [234]:
bool(pd.NA)

TypeError: boolean value of NA is ambiguous

In [235]:
bool(np.NaN)

True

In [236]:
np.NaN == "string"

False

In [305]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1028 entries, 0 to 4128
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   employer    1028 non-null   string 
 1   job_title   1028 non-null   string 
 2   price_from  1028 non-null   Float64
 3   price_to    1028 non-null   Int64  
 4   price_unit  1028 non-null   object 
dtypes: Float64(1), Int64(1), object(1), string(2)
memory usage: 50.2+ KB


In [ ]:
df = df[df['price_unit' ]

In [248]:
df.isna().sum()

employer      0
job_title     0
price_from    0
price_to      0
price_unit    0
dtype: int64

In [274]:
df = df.dropna()

In [282]:
# remove leading and trailing shitespaces
df.price_unit.str.strip()

0       netto (+ VAT) / mies.
1                gross / mth.
2                gross / mth.
3       netto (+ VAT) / godz.
4       netto (+ VAT) / godz.
                ...          
4101             gross / mth.
4122             gross / mth.
4124    netto (+ VAT) / mies.
4126             gross / mth.
4128             gross / mth.
Name: price_unit, Length: 1028, dtype: object

In [310]:

df['new_price_from'] = df.apply(lambda row: row['price_from']*160 if row['price_unit'] == "gross / hr."else row['price_from'], axis=1)
df['new_price_to'] = df.apply(lambda row: row['price_to']*160 if row['price_unit'] == "gross / hr."else row['price_to'], axis=1)

In [312]:
df.head()

,employer,job_title,price_from,price_to,price_unit,new_price_from,new_price_to
0,Prime Engineering Poland,Senior Test Automation Engineer,17000.0,21000,gross / mth.,17000.0,21000
1,Velis RET spółka z ograniczoną odpowiedzialnością,Project Management Specialist,5000.0,7000,gross / mth.,5000.0,7000
2,Velis RET spółka z ograniczoną odpowiedzialnością,Project Manager,7000.0,10000,gross / mth.,7000.0,10000
3,4IT SOLUTIONS sp. z o.o.,Java Fullstack Developer,90.0,130,gross / hr.,14400.0,20800
4,ITFS sp. z o.o.,Tester Automatyzujący,80.0,120,gross / hr.,12800.0,19200


In [313]:
df.nlargest(20, 'new_price_to')

,employer,job_title,price_from,price_to,price_unit,new_price_from,new_price_to
267,ITDS Polska Sp. z o.o.,Java Developer (Micro-Services),55000.0,70000,gross / mth.,55000.0,70000
274,ITDS Polska Sp. z o.o.,Database Developer (Oracle),55000.0,70000,gross / mth.,55000.0,70000
287,ITDS Polska Sp. z o.o.,Senior Java Developer,55000.0,70000,gross / mth.,55000.0,70000
301,ITDS Polska Sp. z o.o.,QA,55000.0,70000,gross / mth.,55000.0,70000
261,ITDS Polska Sp. z o.o.,Senior Backup Engineer,46200.0,54600,gross / mth.,46200.0,54600
1068,ITDS Polska Sp. z o.o.,CMDB Librarian (Splunk),46200.0,54600,gross / mth.,46200.0,54600
1070,ITDS Polska Sp. z o.o.,Splunk Engineer,46200.0,54600,gross / mth.,46200.0,54600
361,ITFS sp. z o.o.,Cassandra Migration Expert,270.0,330,gross / hr.,43200.0,52800
1060,Digital Hub Warsaw at Bayer,IT Enterprise Architect – Application Moderniz...,31000.0,47000,gross / mth.,31000.0,47000
1061,Digital Hub Warsaw at Bayer,Digital Lead - Supply Chain Planning,32000.0,47000,gross / mth.,32000.0,47000
